<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [2]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [3]:
import folium
import pandas as pd

In [4]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [ ]:
## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='red', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:yellow;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [9]:
# Create a folium map object centered at the average location of all launch sites
site_map = folium.Map(location=[launch_sites_df['Lat'].mean(), launch_sites_df['Long'].mean()], zoom_start=5)

# Iterate through the DataFrame and add circles and markers for each launch site
for index, row in launch_sites_df.iterrows():
    coord = [row['Lat'], row['Long']]
    
    # Create a blue circle for each launch site
    circle = folium.Circle(
        coord, radius=1000, color='blue', fill=True
    ).add_child(folium.Popup(row['Launch Site']))
    
    # Create a marker with a label for each launch site
    marker = folium.Marker(
        coord,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color: blue;"><b>%s</b></div>' % row['Launch Site'],
        )
    )
    
    # Add circle and marker to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map
site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [10]:
# Task 2: Mark the success/failed launches for each site on the map
spacex_df.shape


(56, 4)

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [14]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [17]:

# Define a function to set marker_color based on class value
def set_marker_color(class_value):
    if class_value == 1:
        return 'green'
    else:
        return 'red'

# Apply the function to create a new column 'marker_color' based on 'class'
spacex_df['marker_color'] = spacex_df['class'].apply(set_marker_color)

spacex_df.tail()


,Launch Site,Lat,Long,class,marker_color
51,CCAFS SLC-40,28.563197,-80.57682,0,red
52,CCAFS SLC-40,28.563197,-80.57682,0,red
53,CCAFS SLC-40,28.563197,-80.57682,0,red
54,CCAFS SLC-40,28.563197,-80.57682,1,green
55,CCAFS SLC-40,28.563197,-80.57682,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [20]:
marker_cluster = folium.plugins.MarkerCluster()
marker_cluster = MarkerCluster()

# Iterate over each row in spacex_df to add markers
for index, row in spacex_df.iterrows():
    marker = folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=folium.Icon(color=row['marker_color'])  
    )
    marker_cluster.add_child(marker)

# Add the marker cluster to the map
site_map.add_child(marker_cluster)

# Display the map
site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [ ]:
# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [21]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [23]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [24]:
import folium
from folium import DivIcon

# Coordinates for the closest points
launch_site_coords = (34.632834, -120.610745)  # Launch site (VAFB SLC-4E)
closest_city_coords = (34.63321, -120.47504)   # Town coordinates
closest_rail_coords = (34.63208, -120.62473)   # Rail coordinates
closest_highway_coords = (34.63886, -120.45685)  # Highway coordinates
closest_coast_coords = (34.63293, -120.62585)  # Coast coordinates

# Create the site map
site_map = folium.Map(location=launch_site_coords, zoom_start=12)

# Create markers and lines for city, railway, and highway
locations = {
    "Closest City": closest_city_coords,
    "Closest Railway": closest_rail_coords,
    "Closest Highway": closest_highway_coords
}

for name, coords in locations.items():
    distance = calculate_distance(launch_site_coords[0], launch_site_coords[1], coords[0], coords[1])
    
    # Create a marker with distance
    marker = folium.Marker(
        location=coords,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
        )
    )
    
    # Add the marker to the map
    site_map.add_child(marker)
    
    # Draw a line to the launch site
    line = folium.PolyLine(locations=[launch_site_coords, coords], color='blue', weight=1.5, opacity=0.7)
    site_map.add_child(line)

# Display the map
site_map



In [25]:
import folium
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

# Use the provided calculate_distance function
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# Coordinates for the closest points
launch_site_coords = (34.632834, -120.610745)  # Launch site (VAFB SLC-4E)
coast_coords = (34.63293, -120.62585)
rail_coords = (34.63208, -120.62473)
town_coords = (34.63321, -120.47504)
highway_coords = (34.63886, -120.45685)

# Create the site map
site_map = folium.Map(location=launch_site_coords, zoom_start=12)

# Add a marker for the launch site
folium.Marker(
    location=launch_site_coords,
    popup="Launch Site (VAFB SLC-4E)",
    icon=folium.Icon(color="red", icon="rocket", prefix="fa")
).add_to(site_map)

# Define locations with names
locations = {
    "Coastline": coast_coords,
    "Railway": rail_coords,
    "Town": town_coords,
    "Highway": highway_coords
}

# Iterate through all locations to add markers, distance labels, and lines
for name, coords in locations.items():
    distance = calculate_distance(launch_site_coords[0], launch_site_coords[1], coords[0], coords[1])
    
    # Add a marker for each location
    folium.Marker(
        location=coords,
        popup=f"{name}: {distance:.2f} km from launch site",
        icon=folium.Icon(color="purple" if name != "Coastline" else "blue", icon="info-sign")
    ).add_to(site_map)
    
    # Add a distance marker
    folium.Marker(
        location=coords,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{distance:.2f} KM</b></div>'
        )
    ).add_to(site_map)
    
    # Draw a line between the launch site and each location
    folium.PolyLine(locations=[launch_site_coords, coords], color='green' if name == "Coastline" else 'gray', weight=2).add_to(site_map)

# Fit the map to show all markers
site_map.fit_bounds(site_map.get_bounds())

# Save and display the map
site_map.save("launch_site_distances_map.html")
site_map


# Coordinates for the coastline and launch site
coastline_coords = [
    (34.63293, -120.62585),  # Closest coastline point
    (34.632834, -120.610745)  # Launch site (VAFB SLC-4E)
]

# Create a PolyLine object
lines = folium.PolyLine(
    locations=coastline_coords,
    weight=2,  # Line thickness
    color='blue',  # Line color
    opacity=0.7  # Line opacity
)

# Add the PolyLine to your site map
site_map.add_child(lines)

# Display the map
site_map# Coordinates for the coastline and launch site
coastline_coords = [
    (34.63293, -120.62585),  # Closest coastline point
    (34.632834, -120.610745)  # Launch site (VAFB SLC-4E)
]

# Create a PolyLine object
lines = folium.PolyLine(
    locations=coastline_coords,
    weight=2,  # Line thickness
    color='blue',  # Line color
    opacity=0.7  # Line opacity
)

# Add the PolyLine to your site map
site_map.add_child(lines)

# Display the map
site_map*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [44]:
# Coordinates for the coastline and launch site
coastline_coords = [
    (34.63293, -120.62585),  # Closest coastline point
    (34.632834, -120.610745)  # Launch site (VAFB SLC-4E)
]

# Create a PolyLine object
lines = folium.PolyLine(
    locations=coastline_coords,
    weight=2,  # Line thickness
    color='blue',  # Line color
    opacity=0.7  # Line opacity
)

# Add the PolyLine to your site map
site_map.add_child(lines)

# Display the map
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [45]:
import folium
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians


# Coordinates for the closest points
launch_site_coords = (34.632834, -120.610745)  # Launch site (VAFB SLC-4E)
coast_coords = (34.63293, -120.62585)
rail_coords = (34.63208, -120.62473)
town_coords = (34.63321, -120.47504)
highway_coords = (34.63886, -120.45685)

# Create the site map
site_map = folium.Map(location=launch_site_coords, zoom_start=12)

# Add a marker for the launch site
folium.Marker(
    location=launch_site_coords,
    popup="Launch Site (VAFB SLC-4E)",
    icon=folium.Icon(color="red", icon="rocket", prefix="fa")
).add_to(site_map)

# Calculate distance between launch site and coastline
coast_distance = calculate_distance(launch_site_coords[0], launch_site_coords[1], coast_coords[0], coast_coords[1])

# Create and add a marker for the closest coastline point
coast_marker = folium.Marker(
    location=coast_coords,
    popup="Closest Coastline Point",
    icon=folium.Icon(color="blue", icon="water", prefix="fa")
)
coast_marker.add_to(site_map)

# Create and add a distance marker for the coastline
distance_marker = folium.Marker(
    coast_coords,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12px; color:#d35400;"><b>{coast_distance:.2f} KM</b></div>'
    )
)
distance_marker.add_to(site_map)

# Draw a line between launch site and coastline
folium.PolyLine(locations=[launch_site_coords, coast_coords], color='green', weight=2, opacity=0.8).add_to(site_map)

# Add other locations
other_locations = {
    "Railway": rail_coords,
    "Town": town_coords,
    "Highway": highway_coords
}

for name, coords in other_locations.items():
    distance = calculate_distance(launch_site_coords[0], launch_site_coords[1], coords[0], coords[1])
    
    # Create a marker
    marker = folium.Marker(
        location=coords,
        popup=f"{name}: {distance:.2f} km from launch site",
        icon=folium.Icon(color="purple", icon="info-sign")
    )
    marker.add_to(site_map)
    
    # Draw a line to the launch site
    folium.PolyLine(locations=[launch_site_coords, coords], color='gray', weight=1, opacity=0.6).add_to(site_map)

# Fit the map to show all markers
site_map.fit_bounds(site_map.get_bounds())

# Save the map
site_map.save("launch_site_coastline_map.html")

site_map

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
